This Colab shows some example code how to make use of the
[LiT: Zero-Shot Transfer with Locked-image text Tuning](https://arxiv.org/abs/2111.07991)
models in the `big_vision` codebase.

For more information refer to

https://github.com/google-research/big_vision/blob/main/README.md

https://github.com/google-research/big_vision/blob/main/big_vision/configs/proj/image_text/README.md

### Initialize

In [1]:
!git clone --branch=main --depth=1 https://github.com/google-research/big_vision
!cd big_vision && git pull

fatal: destination path 'big_vision' already exists and is not an empty directory.
Already up to date.


In [2]:
!pip install -qr big_vision/big_vision/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [3]:
import sys
bv_path = './big_vision'
if bv_path not in sys.path:
  sys.path.insert(0, bv_path)

%load_ext autoreload
%autoreload 2

In [4]:
from absl import flags
from absl import logging
import tensorflow_datasets as tfds
from google.colab import files

logging.set_verbosity(logging.INFO)

def set_max_height(max_height):
  """Limits scrollable area of output cell to `max_height` pixels."""
  import IPython.display
  IPython.display.display(IPython.display.Javascript('''
    google.colab.output.setIframeHeight(0, true, {maxHeight: %d})
  ''' % max_height))

In [5]:
# Set up Colab  TPUs (if available).
import os
if 'COLAB_TPU_ADDR' in os.environ:
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()
else:
  !nvidia-smi
import jax
jax.devices()

Mon Mar 13 10:56:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

In [6]:
# Uncomment this snippet to access a private GCS bucket with prepared
# TFDS datasets.

# import tensorflow_datasets as tfds
# from google.colab import auth
# auth.authenticate_user()  # Required to access access protected GCS buckets.
# import os
# os.environ['TFDS_DATA_DIR'] = 'gs://tensorflow-datasets/datasets'
# builder = tfds.builder('coco_captions')
# b = next(iter(builder.as_dataset('val')))

### Load training run data

In [7]:
import json
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.io import gfile

def plot_metrics(workdir, regexes, cols=4, cmp={}):
  """Plots metrics matching `regexes` from `workdir`."""
  df = pd.DataFrame([json.loads(line) for line in gfile.GFile(f'{workdir}/big_vision_metrics.txt')])
  df = df.set_index('step')
  ms = []
  for regex in regexes:
    for col in df.columns:
      if col not in ms and re.match(regex, col):
        ms.append(col)
  rows = int(np.ceil(len(ms) / cols))
  _, axs = plt.subplots(rows, cols, figsize=(4*cols, 3*rows))
  if rows == 1: axs = [axs]
  for i, m in enumerate(ms):
    ax = axs[i // cols][i % cols]
    df[m].dropna().plot(ax=ax)
    if m in cmp: cmp[m].dropna().plot(ax=ax)
    ax.set_title(m)
  plt.tight_layout()
  return df

# Reference run using the tiny 80k "coco-captions" TFDS dataset.
df = plot_metrics('gs://vit_models/lit/big_vision/coco_B16B', [
    'training', 'val/loss', 'img/', '.*net2012',
    '.*cifar100', '.*pet', '.*@1$',
])

### Inference

Using an example from the online Demo

https://google-research.github.io/vision_transformer/lit/

In [8]:
!test -f apple-ipod.jpg || wget https://cdn.openai.com/multimodal-neurons/assets/apple/apple-ipod.jpg

labels = [
    'an apple',
    'an ipod',
    'granny smith',
    'an apple with a note saying "ipod"',
    'an adversarial attack',
]

import PIL
import numpy as np
img = np.array(PIL.Image.open('apple-ipod.jpg'))
import matplotlib.pyplot as plt
plt.imshow(img)
img.shape, img.dtype

((322, 322, 3), dtype('uint8'))

In [9]:
!test -f LiT-B16B.npz || gsutil cp gs://vit_models/lit/LiT-B16B.* .

In [11]:
files.view('big_vision/big_vision/configs/proj/image_text/siglip_lit_coco.py')
from big_vision.configs.proj.image_text import siglip_lit_coco as lit_coco
arg = 'txt=bert_base,img=B/16,img_head,init=LiT-B16B.npz'
config = lit_coco.get_config(arg)

<IPython.core.display.Javascript object>

In [12]:
# Initialize template params...
import importlib
import jax.numpy as jnp

model_mod = importlib.import_module(f'big_vision.models.{config.model_name}')

model = model_mod.Model(**config.model)

init_params = [
    jnp.zeros(shape, dtype)
    for shape, dtype in zip(config.init_shapes, config.init_types)
]

params0 = model.init(jax.random.PRNGKey(42), *init_params)['params'].unfreeze()

In [13]:
# ... and load/modify pre-trained params.
from big_vision import utils
# Note that `.load()` is responsible for parameter tree surgery to adapt old
# checkpoints to most recent source code.
params = model_mod.load(params0, 'LiT-B16B.npz', config.model)

INFO:absl:ViT: Loading and fixing VERY old posemb
INFO:absl:ViT: Loading and fixing combined cls+posemb
/content/./big_vision/big_vision/utils.py:593: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  vals, tree_def = jax.tree_flatten(tree)
INFO:absl:
INFO:absl:
INFO:absl:Could not find original BERT checkpoint path 'LiT-B16B.npz:txt/bert_model.ckpt', loading big_vision checkpoint 'LiT-B16B.npz:txt'
INFO:absl:
INFO:absl:


In [14]:
# The preprocessing is optimized for efficiently streaming through TFDS
# datasets - below code runs it separately on the image and every text.
set_max_height(222)

from big_vision.pp import builder as pp_builder
for pp_mod in config.pp_modules:
  importlib.import_module(f'big_vision.pp.{pp_mod}')

pp_str = config.evals.val.pp_fn.replace('decode|', '')
imgs = np.array(pp_builder.get_preprocess_fn(pp_str)({
    'image': img[None],
    'captions/text': np.array(['']),
})['image'])
txts = np.stack([
    pp_builder.get_preprocess_fn(pp_str, log_data=False)({
        'image': img[None],
        'captions/text': np.array([label]),
    })['labels']
    for label in labels
])
imgs.shape, txts.shape

<IPython.core.display.Javascript object>

INFO:absl:Data before pre-processing:
{'image': array([[[[123,  94,  52],
         [123,  96,  53],
         [139, 114,  74],
         ...,
         [ 64,  54,  19],
         [ 60,  54,  22],
         [ 54,  48,  16]],

        [[149, 125,  81],
         [149, 126,  84],
         [152, 130,  89],
         ...,
         [ 69,  55,  26],
         [ 61,  54,  28],
         [ 48,  41,  15]],

        [[152, 133,  91],
         [155, 138,  95],
         [153, 135,  95],
         ...,
         [ 75,  59,  33],
         [ 68,  60,  39],
         [ 45,  37,  16]],

        ...,

        [[112, 101,  73],
         [115, 103,  77],
         [177, 165, 141],
         ...,
         [ 15,   9,   9],
         [  1,   0,   0],
         [  1,   0,   0]],

        [[113,  98,  79],
         [160, 145, 126],
         [191, 175, 159],
         ...,
         [  3,   4,   6],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[113,  98,  79],
         [160, 145, 126],
         [191, 175, 159],


((1, 224, 224, 3), (5, 16))

In [15]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [16]:
zimg, _, _ = model.apply({'params': params}, imgs, None)
_, ztxt, _ = model.apply({'params': params}, None, txts)

In [17]:
probs = jax.nn.softmax((zimg[0] @ ztxt.T * np.exp(params['t'])))
list(zip(labels, probs.tolist()))

[('an apple', 0.0028924213256686926),
 ('an ipod', 1.852169361882261e-06),
 ('granny smith', 1.0219791874988005e-05),
 ('an apple with a note saying "ipod"', 0.9970954656600952),
 ('an adversarial attack', 4.8630912630187595e-08)]

### Run evaluation

Below code runs a minimal version of the `big_vision.tols.eval_only` script.

In [18]:
files.view('big_vision/big_vision/tools/eval_only.py')
from big_vision.tools import eval_only

<IPython.core.display.Javascript object>

In [19]:
!test -f LiT-B16B.npz || gsutil cp gs://vit_models/lit/LiT-B16B.* .

In [20]:
from big_vision.configs.proj.image_text import lit_coco
arg = 'txt=bert_base,img=B/16,img_head,init=LiT-B16B.npz'
config = lit_coco.get_config(arg)

In [21]:
# From all the pre-defined evaluators...
set_max_height(222)
config.evals

<IPython.core.display.Javascript object>

coco:
  data:
    name: coco_captions
    split: val
  log_steps: 500
  pp_fn: decode|resize(224)|value_range(-1,1)|flatten|bert_tokenize(inkey="captions/text",
    max_len=16, vocab_path="LiT-B16B.txt")|keep("image", "labels")
  type: proj.image_text.contrastive
  use_global_batch: true
disclf:
  log_steps: 500
  pp_img: resize(224)|value_range(-1,1)
  pp_txt: bert_tokenize(inkey="texts", max_len=16, vocab_path="LiT-B16B.txt")
  prefix: z/0shot/
  type: proj.image_text.discriminative_classifier
imagenet:
  data:
    name: imagenet2012
    split: validation
  log_steps: 500
  pp_fn: decode|resize(224)|value_range(-1,1)|clip_i1k_label_names|bert_tokenize(inkey="labels",
    max_len=16, vocab_path="LiT-B16B.txt")|keep("image", "labels")
  type: proj.image_text.contrastive
  use_global_batch: true
retrieval_coco:
  dataset: coco_captions
  log_steps: 500
  pp_img: resize(224)|value_range(-1, 1)
  pp_txt: bert_tokenize(inkey="texts", max_len=16, vocab_path="LiT-B16B.txt")
  prefix: z/retr/

In [22]:
# ... run a single zeroshot discriminative classifier on pets.
config.evals = {
    'disclf': {
        **config.evals.disclf,
        'dataset_names': ['oxford_iiit_pet'],
        'dataset_overrides': (),
    },
}
config.evals

disclf:
  dataset_names:
  - oxford_iiit_pet
  dataset_overrides: !!python/tuple []
  log_steps: 500
  pp_img: resize(224)|value_range(-1,1)
  pp_txt: bert_tokenize(inkey="texts", max_len=16, vocab_path="LiT-B16B.txt")
  prefix: z/0shot/
  type: proj.image_text.discriminative_classifier

In [23]:
# Prepare pets dataset.
tfds.builder('oxford_iiit_pet').download_and_prepare()

INFO:absl:Load dataset info from /root/tensorflow_datasets/oxford_iiit_pet/3.2.0
INFO:absl:Reusing dataset oxford_iiit_pet (/root/tensorflow_datasets/oxford_iiit_pet/3.2.0)


In [24]:
workdir = 'lit_coco_B16B_eval'
!mkdir $workdir
flags.FLAGS.workdir = workdir
config.input.batch_size = 512
flags.FLAGS.config = config

mkdir: cannot create directory ‘lit_coco_B16B_eval’: File exists


In [25]:
# Should run in ~5 minutes on a T4 GPU...
set_max_height(444)
eval_only.main([])

<IPython.core.display.Javascript object>

INFO:absl:Workdir: lit_coco_B16B_eval
INFO:absl:NOTE: Initializing proj.image_text.two_towers model...
INFO:absl:[0] z/secs/init = 53.88540307900007
INFO:absl:TIMING[z/secs/init]: 53.88540307900007
INFO:absl:
init params
+-------------------------------------------------------------------------------------------------------+------------------+------------+-----------+----------+
| Name                                                                                                  | Shape            | Size       | Mean      | Std      |
+-------------------------------------------------------------------------------------------------------+------------------+------------+-----------+----------+
| img/Transformer/encoder_norm/bias                                                                     | (768,)           | 768        | 0.0       | 0.0      |
| img/Transformer/encoder_norm/scale                                                                    | (768,)           | 768       

INFO:absl:Data after pre-processing:
{'image': <tf.Tensor 'add:0' shape=(224, 224, 3) dtype=float32>, 'label': <tf.Tensor 'args_2:0' shape=() dtype=int64>}
INFO:absl:Data before pre-processing:
{'label': <tf.Tensor 'args_0:0' shape=() dtype=int64>, 'texts': <tf.Tensor 'args_1:0' shape=() dtype=string>}
INFO:absl:Data after pre-processing:
{'label': <tf.Tensor 'args_0:0' shape=() dtype=int64>, 'labels': <tf.Tensor 'strided_slice_4:0' shape=(16,) dtype=int32>}
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
INFO:absl:Initialized evaluator in 3.0 seconds
INFO:absl:NOTE: disclf evaluation step 0...
INFO:absl:Starting text embedding...
INFO:absl:Compiled text embeddings in 7.6s
INFO:absl:Embedded oxford_iiit_pet text in 7 steps - ...[512 512 512 512 512 437   0]
INFO:absl:Totalling 2997 text in 2.6s
INFO:absl:Total texts embeddings size 9.2

In [26]:
# ... and yield a final 81% accuracy.
import json
json.loads(open(flags.FLAGS.workdir + '/big_vision_metrics.txt').readline())

{'step': 0,
 'z/0shot/oxford_iiit_pet_accuracy': 0.8103025347506132,
 'z/secs/eval/disclf': 73.91012993699997}